<a href="https://colab.research.google.com/github/Baah134/Baah134/blob/main/Whisper/Whisper_Transcription_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai-whisper jiwer pandas

import os
import pandas as pd
import glob
import whisper
from jiwer import wer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.2/253.2 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.0 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803373 sha256=a8c7497afe676ea9516db4343b6d9bb475b7a49dedd4140bdd85f13c5d642b07
  Stored in directory: /root/.cache/pip/wheels/2f/f2/ce/6eb23db4091d026238ce76703bd66da60b969d70bcc81d5d3a
Successfully built openai-whisper


In [2]:
from google.colab import drive
drive.mount('/content/drive')

audio_folder = '/content/drive/MyDrive/DeepLearning/Whisper/my_data/audio'
csv_file = "/content/drive/MyDrive/DeepLearning/Whisper/my_data/transcriptions.csv"

Mounted at /content/drive


In [3]:
from tqdm import tqdm

In [4]:
df = pd.read_csv(csv_file)

# Dictionary {audio file name (without extension): transcription}
transcriptions = {os.path.splitext(row["filename"])[0]: row["transcription"] for _, row in df.iterrows()}

# ============================
# STEP 2: LOAD AUDIO FILES
# ============================

# Get all audio files (wav, m4a, etc.)
audio_files = glob.glob(os.path.join(audio_folder, "*"))

# ============================
# STEP 3: INITIALIZE WHISPER
# ============================
model = whisper.load_model("large")  # "small", "medium", "large" for better accuracy

# ============================
# STEP 4: PROCESS EACH AUDIO FILE
# ============================
matched_data = []  # Store results
total_wer = 0  # Sum of all WERs
valid_files = 0  # Count files with transcriptions

for audio_path in tqdm(audio_files, desc="Processing Audios"):
    file_name = os.path.basename(audio_path)  # Extract filename (e.g., "sample_001.wav")
    file_stem = os.path.splitext(file_name)[0]  # Remove extension (e.g., "sample_001")

    # ✅ Ensure the transcription exists
    ground_truth = transcriptions.get(file_stem)
    if ground_truth is None:
        continue  # Skip if no matching transcription

    # ✅ Transcribe audio using Whisper
    result = model.transcribe(audio_path, fp16 = False)
    whisper_transcription = result["text"].strip()

    # ✅ Compute Word Error Rate (WER)
    error_rate = wer(ground_truth, whisper_transcription)

    # ✅ Store results
    matched_data.append((file_name, whisper_transcription, ground_truth, error_rate))
    total_wer += error_rate
    valid_files += 1

# ============================
# STEP 5: COMPUTE AVERAGE WER
# ============================
average_wer = total_wer / valid_files if valid_files else 0

# ============================
# STEP 6: SAVE RESULTS TO CSV
# ============================
output_csv = "whisper_results.csv"
df_results = pd.DataFrame(matched_data, columns=["Filename", "Whisper Transcription", "Ground Truth", "WER"])
df_results.to_csv(output_csv, index=False)

# Print Results
print(df_results)
print(f"\n✅ Average WER: {average_wer:.4f}")

100%|█████████████████████████████████████| 2.88G/2.88G [00:53<00:00, 57.9MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

KeyboardInterrupt: 